In [60]:
import sqlite3
conn = sqlite3.connect('/mnt/external-images-pvc/quantmap/qm_pr01.sqlite')
c = conn.cursor()
#c.execute("SELECT name FROM sqlite_master WHERE type='table';")
#List tables
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in c.fetchall():
    print (table)

('stitch_protchem_man',)
('string_protlink_man',)


In [61]:
# Find the number of brances from the newick tree and to change the header to CID numbers
# Total number of brances for 5338 data is 1489
try:
    open_file = open("newick_format.txt","r").readlines()
    newick_format = ""
    for entry in open_file:
        newick_format += entry[:-1]

    total_branches = newick_format.count("(") - 1
    print ("Total number of branches = " + str(total_branches))

    newick_format = newick_format.replace("CID","")
    write_newick = open("newick_format_output.txt","w")
    write_newick.write(newick_format)
    write_newick.close()
except:
    print ("Total number of branches = 1489")

Total number of branches = 1489


In [62]:
# Read the distance matrix and replace the header to CID numbers
import pandas as pd
distance_matrix = "distance_matrix_qmap_chemicals.txt"
output_matrix = distance_matrix[:-4] + "_out.csv"
open_file = open(distance_matrix,"r").readlines()
distance_matrix = {}
output = ""
for count,line in enumerate(open_file):
    if count == 0:
        heading_list = line.split(",")
        output += "," + line
    else:
        output += heading_list[count-1].strip() + "," + line
    
output = output.replace("CID","")
out_file = open(output_matrix,"w")
out_file.write(output)
out_file.close()
m = pd.read_csv(output_matrix,index_col=0)

In [106]:
# Distance matrix clustering using tsne

from sklearn.manifold import TSNE
import numpy as np
m1 = m.copy()

kmean = ""
kmean = TSNE(n_components=3,n_jobs=8,metric="precomputed",method="exact").fit_transform(m1)
kmean.shape

(18, 3)

In [107]:
kmean

array([[ -74.540955,   12.989391,  -48.282406],
       [ -40.02215 , -235.76843 ,   14.563934],
       [ -11.617976,   94.44393 , -250.69688 ],
       [ -28.796268, -199.16653 , -206.40216 ],
       [ 264.9761  ,  -35.141254,  200.39523 ],
       [ 108.01347 ,  139.03836 ,  140.45038 ],
       [ 116.616196, -249.13199 ,  197.95352 ],
       [-122.48972 ,  165.6543  ,  128.49413 ],
       [-216.01805 , -126.98593 ,  119.4317  ],
       [-282.59772 ,   93.482315,   -8.02976 ],
       [ 130.36319 ,  -59.704636, -150.03822 ],
       [ -44.917446,  -30.252108,  231.37363 ],
       [ 250.43822 ,   96.00705 ,  -51.439125],
       [-229.41696 , -127.30476 , -123.81338 ],
       [ 109.171005,  -78.22495 ,   58.63421 ],
       [-403.3331  ,   26.835087, -181.21281 ],
       [  18.847454,  232.58621 ,  -56.4355  ],
       [ 184.57367 , -270.93555 , -276.34888 ]], dtype=float32)

In [118]:
# Get CID and respective cluster
cid_cluster_kmean = {}
for cid,cluster in zip(cid_list,result):
    cid_cluster_kmean[cid] =  cluster

In [119]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering as AC

kmean = ""
kmean= KMeans(n_clusters=5)
kmean.fit(X_embedded)
result= kmean.labels_
m1['label']= result
cid_cluster = m1['label']

In [51]:
# Get the CID and label of it
# Get smiles from the database
# Replace cid with smiles
# Output to a file

index = cid_cluster.index
cid_list = list(index)
cid_list = list(map(int, cid_list))

connect = sqlite3.connect('/mnt/external-images-pvc/quantmap/qm_chem_fix.sqlite')
d = connect.cursor()
d.execute("select cid,smiles from stitch_chem where cid in ("+str(cid_list)[1:-1]+");")
cid_smiles_list = d.fetchall()

cid_smiles_dict = {}
for entry in cid_smiles_list:
    cid_smiles_dict[entry[0]] = entry[1]
    
output = open("ML_input_5338.txt","w")
cluster_count = {}
for cid,cluster in zip(cid_list,cid_cluster):
    try:
        output.write(str(cid_smiles_dict[cid]) + "\t" + str(cluster) + "\n")
        if cluster not in cluster_count:
            cluster_count[cluster] = 1
        else:
            cluster_count[cluster] += 1
    except:
        pass
output.close()

In [52]:
count = sum([1 for cluster in cluster_count if cluster_count[cluster] == 1])
print ("Clusters with only one label = " + str(count))

Clusters with only one label = 0


In [123]:
# Frequency of the clusters counts. eg, there are ones and the count is 100, twos count is 29 etc

cluster_count_frequency = {}
for cluster in cluster_count:
    if cluster_count[cluster] not in cluster_count_frequency:
        cluster_count_frequency[cluster_count[cluster]] = 1
    else:
        cluster_count_frequency[cluster_count[cluster]] += 1

{k: v for k, v in sorted(cluster_count_frequency.items(), key=lambda item: item[0])}
# Cluster frequency and its count (or number of occurances)

{740: 1, 861: 1, 880: 1, 925: 1, 1817: 1}